In [1]:
import os

In [2]:
PERPLEXITY_API_KEY = os.getenv("Perplexity_API_Key")
GOOGLE_GEMINI_API_KEY = os.getenv("Gemini_API_Key")

In [3]:
import requests

In [4]:
query = "What are the best payment gateways for developers in the UK?"

In [7]:
chat_url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "sonar-pro",
    "messages": [
        {
            "role": "system",
            "content": "Be precise and concise."
        },
        {
            "role": "user",
            "content": query
        }
    ]
}
headers = {
    "Authorization": "Bearer " + PERPLEXITY_API_KEY,
    "Content-Type": "application/json"
}

chat_response = requests.post(chat_url, json=payload, headers=headers)

print(chat_response)

<Response [200]>


In [ ]:
chat_response.json()

In [13]:
from google import genai

In [ ]:
perpelixity_response_text = chat_response.json()['choices'][0]['message']['content']
perpelixity_response_text

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

In [21]:
target_client = "Stripe"
gemini_prompt = f"""
You are an expert data extractor.

The following text delimited by triple askticks is a user query for Perplexity AI:

***
{query}
***

And the following text delimited by triple backticks is the response from Perplexity AI for the query above:

```
{perpelixity_response_text}
```

Your task is to determine if the target client '{target_client}' is mentioned in the response from Perplexity AI, by following these steps:
Step 1. Check if the target client '{target_client}' is mentioned in the response.
Step 2. Extract all the competitors in the response.
Step 3. If client '{target_client}', determine the rank position of the target client '{target_client}' among the competitors extracted, using the following rule:
    - If the target client '{target_client}' is mentioned and a ranking is explicitly list in the response, such as "1. PayPal, 2. Stripe", return the ranking of the target client listed.
    - If the target client '{target_client}' is mentioned but no explicit ranking is provided, return 0 for rank position.
    - If the target client '{target_client}' is not mentioned, return None for rank position.


Return your response as a single JSON object which strictly following the JSON schema given in this request
""".strip()


In [22]:
class RankingResponse(BaseModel):
    is_visisble: bool = Field(
        description=f"Whether if the target client '{target_client}' is mentioned in the response"
    ),
    competitors: List[str] = Field(
        description="List of competitors extracted from the response. If none, return an empty list."
    )
    rank_position: int = Field(
        description="Rank position of the target client among competitors following the ranking rule described in Step 3 in the prompt."
    )

In [24]:
client = genai.Client()

gemini_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=gemini_prompt,
    config={
        "response_mime_type": "application/json",
        "response_json_schema": RankingResponse.model_json_schema(),
    },
)

print(gemini_response.text)

{"is_visisble":true,"competitors":["Stripe","Adyen","Braintree (PayPal)","Square","Checkout.com","Opayo (Sage Pay)"],"rank_position":1}
